In [169]:
import numpy as np
import pandas as pd
import datetime
from numpy.linalg import inv
from pandas_datareader import data as pdr
import yfinance as yfin
#import matplotlib.pyplot as plt
#from scipy.optimize import minimize
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

yfin.pdr_override()


In [170]:
def calculate_daily_returns(adj_close):
    """
    Calculates the daily average returns for a given dataset of adjusted closing prices.
    
    Parameters:
    -----------
    adj_close : pandas.DataFrame
        A DataFrame containing the adjusted closing prices of one or more stocks.
    
    Returns:
    --------
    pandas.DataFrame
        A DataFrame containing the daily average returns.
    """
    # Calculate the daily returns using the adjusted closing prices
    daily_returns = (adj_close / adj_close.shift(1)) - 1
    
    return daily_returns

In [171]:
def portfolio_covariance(data):
    """
    Calculates the yearly average portfolio covariance for a given dataset.
    
    Parameters:
    -----------
    data : pandas.DataFrame
        A DataFrame containing the daily values of assets in the portfolio.
    
    Returns:
    --------
    pandas.DataFrame
        A DataFrame containing the yearly average portfolio covariance.
    """
    # Calculate the daily returns using the data_returns function
    daily_returns = calculate_daily_returns(data)
    
    # Calculate the yearly portfolio covariance
    yearly_port_cov = daily_returns.cov() * 252
    
    return yearly_port_cov

#### Markowitz Portfolio Theory Function

In [184]:
def generate_combinations(tickers, start_date, end_date):
    """
    Generates 1000 random combinations of the given stocks and calculates the average annual return and 
    risk for each combination.
    
    Parameters:
    -----------
    tickers : list
        A list of strings representing the stock symbols of the stocks to be analyzed.
    start_date : str
        A string representing the start date in 'YYYY-MM-DD' format for the data to be retrieved.
    end_date : str
        A string representing the end date in 'YYYY-MM-DD' format for the data to be retrieved.
    
    Returns:
    --------
    pandas.DataFrame
        A DataFrame containing the random weight combinations and the average annual return and risk
        for each combination.
    """
    # Load data and create random weights DataFrame
    data = pd.DataFrame()
    data = pdr.get_data_yahoo(tickers, start_date, end_date)['Adj Close']
    weights_array = pd.DataFrame(columns=tickers)

    # Empty arrays for return and risk columns
    port_return = []
    port_risk = []
  
    # Generate 1000 random combinations for the portfolio
    for w in range(1000):
        # Get random weights for each stock
        weights = np.random.random(len(tickers)) 

        # Normalize the weights to ensure they sum to 1
        weights /= np.sum(weights) 
        
        # Insert weights array to a DataFrame called weights_array
        weights_array.loc[len(weights_array)] = weights.tolist()  
        
        # Calculate average annual return and store it in port_return array
        port_return.append(np.sum(weights * (calculate_daily_returns(data).mean()*252)))
        
        # Calculate average annual standard deviation/risk and store it in port_risk array
        port_risk.append(np.sqrt(np.dot(weights.T, np.dot(portfolio_covariance(data), weights)))) 

    # Create DataFrame called portfolios to store risk and return
    portfolios = pd.DataFrame({'Return':port_return, 'Risk': port_risk}) 
    portfolios['Sharpe Ratio'] = (portfolios['Return']-0.00304)/portfolios['Risk']
    weights_array.columns = tickers
        
    # Concatenate DataFrame weights_array and portfolios
    result = pd.concat([weights_array, portfolios], axis=1)
    
    return result

In [185]:
#Calculation Minimum Variance Portfolio with Lagrange multiplier and Cramer's Rule based on Modul Portfolio Selection from Prof. Dr. Andreas Görg
def get_minimum_variance_portfolio(Ticker, start_date, end_date):
    data = pd.DataFrame()
    data = pdr.get_data_yahoo(Ticker, start_date, end_date)['Adj Close']

    # Variables
    Mu = (calculate_daily_returns(data).mean() * 252).to_numpy()
    Sigma = (portfolio_covariance(data)).to_numpy()
    One = [1 for x in range(len(Mu))]
    One_arr = np.array(One)
    Sigma_inverse = inv(Sigma)

    a = np.dot(np.dot(Mu.T, Sigma_inverse), Mu)
    b = np.dot(np.dot(Mu.T, Sigma_inverse), One_arr)
    c = np.dot(np.dot(One_arr, Sigma_inverse), One_arr)

    Return_x = b / c
    Risk_x = 1 / np.sqrt(c)

    weights = (np.dot((Return_x * c - b) * Sigma_inverse, Mu) + (np.dot((a - Return_x * b) * Sigma_inverse, One_arr))) / (a * c - b ** 2)

    Min_Port = np.array([Return_x, Risk_x])

    Data = np.concatenate((weights, Min_Port))

    Columns = Ticker
    Columns.append('Return')
    Columns.append('Risk')

    Min_Varianz_Data = pd.DataFrame(columns=Columns)
    Min_Varianz_Data.loc[len(Min_Varianz_Data)] = Data
    Min_Varianz_Data['Sharpe Ratio'] = (Min_Varianz_Data['Return']-0.00304)/Min_Varianz_Data['Risk']

    return Min_Varianz_Data


#### Calling Function

In [186]:
stocks = ['AAPL','TSLA','MSFT','GOOG']
start = datetime.datetime(2018,1,1)
end = datetime.datetime.now()

In [187]:
my_combination = generate_combinations(stocks,start,end)

[*********************100%***********************]  4 of 4 completed


In [188]:
my_combination

,AAPL,TSLA,MSFT,GOOG,Return,Risk,Sharpe Ratio
0,0.196923,0.112020,0.019371,0.671686,0.507251,0.497187,1.014126
1,0.156642,0.033971,0.462453,0.346934,0.404828,0.365052,1.100632
2,0.604545,0.075965,0.243246,0.076244,0.319367,0.309589,1.021765
3,0.022055,0.143831,0.529320,0.304794,0.375084,0.350013,1.062943
4,0.481753,0.069642,0.295159,0.153446,0.343235,0.317285,1.072209
...,...,...,...,...,...,...,...
995,0.107545,0.216020,0.205552,0.470884,0.425478,0.406755,1.038556
996,0.001492,0.375616,0.385178,0.237714,0.326168,0.328865,0.982555
997,0.024277,0.324942,0.021808,0.628972,0.464966,0.473736,0.975072
998,0.331332,0.102678,0.350999,0.214991,0.356761,0.326281,1.084100


In [189]:
Sharpe_max = my_combination[my_combination['Sharpe Ratio']== my_combination['Sharpe Ratio'].max()]
Sharpe_max

,AAPL,TSLA,MSFT,GOOG,Return,Risk,Sharpe Ratio
757,0.433397,0.000507,0.288372,0.277724,0.391839,0.34859,1.115348


In [190]:
Min_Var_Portfolio = get_minimum_variance_portfolio(stocks,start,end)
Min_Var_Portfolio

[*********************100%***********************]  4 of 4 completed


,AAPL,TSLA,MSFT,GOOG,Return,Risk,Sharpe Ratio
0,0.281672,0.365056,0.374665,-0.021393,0.246355,0.291865,0.833654


In [191]:
#More Simple Version of Minimum Variance

# Find index of row with minimum risk
min_risk_idx = my_combination['Risk'].idxmin()

# Subset DataFrame to only include rows with minimum risk
min_risk_df = my_combination[my_combination['Risk'] == my_combination.loc[min_risk_idx, 'Risk']]

# Find index of row with maximum return among rows with minimum risk
optimal_return_idx = min_risk_df['Return'].idxmax()

# Subset DataFrame to only include row with both minimum risk and maximum return
optimal_portfolio = min_risk_df[min_risk_df['Return']==min_risk_df.loc[optimal_return_idx,'Return']]

In [192]:
# Reformat data
def reformat_Data(Data):
     Data_1 = Data.copy()
     # Convert percentage columns to string with two decimal places and add '%'
     Data_1.iloc[:,:-3] = (Data_1.iloc[:,:-3] * 100).round(2).astype(str) + "%"
     # Round numerical columns to two decimal places and multiply by 100
     # Data_1.iloc[:,3:5] = (Data_1.iloc[:,3:5] * 100).round(2)
     return Data_1



In [193]:
Scatter_Data = reformat_Data(my_combination)
Minimum_Variance_Data = reformat_Data(Min_Var_Portfolio)
Sharpe_max_Data = reformat_Data(Sharpe_max)
Optimal_Portfolio_Data = reformat_Data(optimal_portfolio)

In [182]:
sorted = Scatter_Data.sort_values(by=['Risk'])
efficient = sorted.groupby(np.arange(len(sorted))//10).max()

#### Visualization

In [195]:
fig = go.Figure()

# Add Portfolio Combinations trace
fig.add_trace(
    go.Scatter(
        x=Scatter_Data['Risk'],
        y=Scatter_Data['Return'], 
        mode='markers',
        marker=dict(
            color=Scatter_Data['Sharpe Ratio'], 
            colorscale='Blues',
            colorbar=dict(
                title='Sharpe Ratio',
                orientation='h'
            ),
            size=10,
            opacity=0.8
        ),
        name='Portfolio Combinations',
        hovertemplate='<b>Sharpe Ratio</b>: %{marker.color:.2f}<br><b>Risk</b>: %{x:.2%}<br><b>Return</b>: %{y:.2%}<extra></extra>'
    )
)

# Add Minimum Variance Portfolio trace
fig.add_trace(
    go.Scatter(
        x=Minimum_Variance_Data['Risk'],
        y=Minimum_Variance_Data['Return'], 
        mode='markers',
        marker=dict(
            color=Minimum_Variance_Data['Sharpe Ratio'], 
            colorscale='reds',
            size=15,
            symbol='x'
        ),
        name='Minimum Variance Portfolio',
        hovertemplate='<b>Minimum Variance Portfolio</b><br><b>Sharpe Ratio</b>: %{marker.color:.2f}<br><b>Risk</b>: %{x:.2%}<br><b>Return</b>: %{y:.2%}<extra></extra>'
    )
)

# Add Maximum Sharpe Portfolio trace
fig.add_trace(
    go.Scatter(
        x=Sharpe_max_Data['Risk'],
        y=Sharpe_max_Data['Return'], 
        mode='markers',
        marker=dict(
            color=Sharpe_max_Data['Sharpe Ratio'], 
            colorscale='viridis',
            size=15,
            symbol='x'
        ),
        name='Max Sharpe Portfolio',
        hovertemplate='<b>Maximum Sharpe Ratio Portfolio</b><br><b>Sharpe Ratio</b>: %{marker.color:.2f}<br><b>Risk</b>: %{x:.2%}<br><b>Return</b>: %{y:.2%}<extra></extra>'
    )
)

# Add Minimum Variance Portfolio 2 trace
fig.add_trace(
    go.Scatter(
        x=Optimal_Portfolio_Data['Risk'],
        y=Optimal_Portfolio_Data['Return'], 
        mode='markers',
        marker=dict(
            color=Optimal_Portfolio_Data['Sharpe Ratio'], 
            colorscale='tealrose',
            size=15,
            symbol='x'
        ),
        name='Minimum Variance Portfolio 2',
        hovertemplate='<b>Optimal Portfolio</b><br><b>Sharpe Ratio</b>: %{marker.color:.2f}<br><b>Risk</b>: %{x:.2%}<br><b>Return</b>: %{y:.2%}<extra></extra>'
    )
)

# Add Efficient Frontier Line trace
# fig.add_trace(
#     go.Scatter(
#         x=efficient['Risk'],
#         y=efficient['Return'],
#         mode='lines',
#         line=dict(
#             color='black',
#             width=3,
#             dash='dash'
#         ),
#         name='Efficient Frontier Line',
#         hovertemplate='<b>Imperfect Efficient Frontier</b><br><b>Risk</b>: %{x:.2%}<br><b>Return</b>: %{y:.2%}<extra></extra>'
#     )
# )

fig.update_layout(
    title = "Portfolio Combination of " + " - ".join(Scatter_Data.columns[:-3]),
    xaxis_title="Risk (%)",
    yaxis_title="Return (%)",
    font=dict(
        family="Bahnschrift",
        size=13,
        color="RebeccaPurple"
    ),
    coloraxis_colorbar_title='Sharpe Ratio',
    height=600,  # increase height of the plot
    width=1000
)

fig.show()